<a href="https://colab.research.google.com/github/SedoyChloric/work_in_collab/blob/test/MiniSOM_Clusterisation%20%D0%BD%D0%B0%D1%87%D0%B0%D0%BB%D0%BE%2006%2C04%2C2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from IPython import get_ipython
get_ipython().system('pip install minisom==2.3.5')
import matplotlib.pyplot as plt
from minisom import MiniSom
import numpy as np
from google.colab import auth
auth.authenticate_user()
from pylab import plot,axis,show,pcolor,colorbar,bone

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

  Preparing metadata (setup.py) ... done
  Created wheel for minisom: filename=MiniSom-2.3.5-py3-none-any.whl size=12031 sha256=51172d5c03dc9079eda1a46285bb07ada536e1c14febdb4e9430358f0c42161d
  Stored in directory: /root/.cache/pip/wheels/19/db/95/5e53bc2b88a328217fdf9f2886cafbe86b0df274f4b601f572
Successfully built minisom


In [ ]:
alphabet = np.array(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n',
           'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'])

In [ ]:
def get_data_from_googlesheet(google_spreedsheet):
  aerogels_properties = np.array(google_spreedsheet.worksheet("Данные по аэрогелям").get_all_values()) #Лист преобразуется в массив
  name_of_properties = aerogels_properties[0][2:-1] # Получаем наименования свойств (со второго до предпоследнего)
  aerogels_properties = np.transpose(aerogels_properties) #Транспонируем
  id = aerogels_properties[1][1:] #Получаем лист наименований
  source = aerogels_properties[-1][1:] #Получаем лист сурсов
  prop = aerogels_properties[2:-1][0:] #Получаем лист свойств, который нужно будет снова транспонировать
  return id, source, np.transpose(np.delete(prop, 0, 1).astype('float64')), name_of_properties

def to_normalize_except_material(array):
  min_val = np.min(array[:, 1:])
  max_val = np.max(array[:, 1:])
  normed_array = (array[:, 1:] - min_val) / (max_val - min_val)
  return np.column_stack((array[:, 0], normed_array))

def add_neighbors(data, point_index, eps):
  neighbors = np.array([])
  for i in range(len(data)):
    if np.linalg.norm(data[i]-data[point_index]) <= eps:
      neighbors = np.append(neighbors, i)
  return neighbors.astype('int32')

def release_dbscan(data, eps, min_points):
  cluster_id = 0
  labels = np.zeros(len(data), dtype='int32')
  for point in range(len(data)):
    if labels[point] != 0:
      continue
    neighbors = add_neighbors(data, point, eps)
    if len(neighbors) < min_points:
      labels[point] = -1
      continue
    labels[point] = cluster_id + 1
    seed = neighbors
    while len(seed) > 0:
      current_point = seed[0]
      seed = np.delete(seed, 0)
      if labels[current_point] == -1:
        labels[current_point] = cluster_id +1
      if labels[current_point] != 0:
        continue
      labels[current_point] = cluster_id + 1
      new_neighbors = add_neighbors(data, current_point, eps)
      if len(new_neighbors) >= min_points:
        seed = np.append(seed, new_neighbors)
    cluster_id += 1
  return labels

def calculate_column(number, alphabet=alphabet):
  column = alphabet[number%len(alphabet)-1]
  while number//len(alphabet) != 0:
    number = number//len(alphabet)
    column = alphabet[number%len(alphabet)-1] + column
  return column

def pack_clusters(data, labels, minimum_points):
  packed_clusters = labels.astype(str)
  cluster_id = 0
  packed_clusters[labels == -1] = "Шум"
  for i in np.unique(labels):
    cluster = data[labels == i]
    packed_clusters[labels == i] = f"Кластер №{cluster_id+1}"
    cluster_id +=1
  return packed_clusters

def to_list(array):
  return array.reshape(-1, 1).tolist()

def prepare_googlesheet(id_array, source_array, property_values, name_of_properties, worksheet):
  worksheet.update(to_list(np.arange(len(id_array)+1)), 'a1:a')
  worksheet.update(to_list(id_array), 'b2:b')
  worksheet.update([name_of_properties.tolist()], 'c1')
  worksheet.update(property_values.tolist(), 'c2')
  worksheet.update_acell('b1', "Наименование точки")
  worksheet.update_acell(f'{calculate_column(len(name_of_properties)+3)}1', "Кластеризация")
  return

def import_result_in_googlesheet(packed_clusters, worksheet, sigma, learning_rate):
  column = worksheet.find('Кластеризация').col
  worksheet.update_acell(f'{calculate_column(column)}1', f's={sigma}; lr={learning_rate}')
  worksheet.update_acell(f'{calculate_column(column+1)}1', 'Кластеризация')
  worksheet.update(to_list(packed_clusters), f'{calculate_column(column)}2:{calculate_column(column)}')
  return

def batch_clusterizarion(data, normed_data, start_eps, minimum_points, worksheet):
  step = 0.01
  end_eps = np.round(start_eps*10, 4)
  i = start_eps
  while i <= end_eps:
    labels = release_dbscan(normed_data, i, minimum_points) ####
    packed_clusters = pack_clusters(data, labels, minimum_points)
    import_result_in_googlesheet(packed_clusters, worksheet, np.round(i, 4), minimum_points)
    i += np.round(step, 4)
  return

In [ ]:
aerogel_spreedsheet = gc.open_by_key('1PMc-dFmc5srbDXqIfrrWNw_HHRrXcHkL5w0bQ5Bb1HU')
experiment_spreadsheet = gc.open_by_key('1etXfria78osnRn9U6QjbXlWe2KgljOp7mreyIEdNV9U')
id_array, source_array, data, name_of_properties = get_data_from_googlesheet(aerogel_spreedsheet)

In [ ]:
print("Есть на просторах интернета одна гугл-таблица, в которой собрано немного про характеристики аэрогелей.\n"
  "В этой прекрасной гугл-таблице насчитываются характеристики уже", len(id_array), "аэрогелей\n"
  "Эта программа может разбивать аэрогели на группы при помощи магии, которая зовётся MiniSOM\n"
  "Магия работает по таким характеристикам, как", name_of_properties)
while True:
  print("1. Кронк, жми на рычаг просмотра данных!"
  "\n2. Кронк, жми на рычаг кластеризации!"
  "\n3. Кронк, жми на рычаг в лабораторию!")
  try:
    choice = int(input("Нажмите на рычаг: "))
    if choice == 1:
      for i in range(len(id_array)-1):
        print("Точка", id_array[i], "с координатами:\n", data[i])
    elif choice == 2:
      try:
        sheet_name = input("Результаты будут импортированы в гугл-таблицу. \nИмя листа будет: ")
        worksheet = experiment_spreadsheet.add_worksheet(title=sheet_name, rows=len(id_array)+1, cols=100)
        prepare_googlesheet(id_array, source_array, data, name_of_properties, worksheet)
      except gspread.exceptions.APIError:
        worksheet = experiment_spreadsheet.worksheet(sheet_name)
      while True:
        print("Задать сигму\n"
              "Одного единого универсального значения нет!\n")
        while True:
            try:
              sigma = float(input("Введите сигму: "))
              break
            except ValueError:
              print("Я не знаю таких цифр")
        print("Задать скорость обучения\n",
              "1. Использовать 3\n",
              "2. Задать число самому\n")
        choice_2 = input("Введите опцию: ")
        while True:
          if choice_2 == '1':
            learning_rate = 0.5
            break
          elif choice_2 == '2':
            while True:
              try:
                learning_rate = int(input("Введите минимальное количество точек (Рекомендуется брать от 5 до 2): "))
                break
              except ValueError:
                print("Я не знаю таких цифр")
          else:
            print("Я не знаю таких цифр")
        #normed_data = to_normalize_except_material(data)
        normed_data = to_normalize_except_material(data)
        som = MiniSom(1, len(normed_data[:]), len(normed_data[0, :]), sigma=sigma, learning_rate=learning_rate)
        som.train_batch(normed_data, 100)
        winner_coordinates = np.array([som.winner(x) for x in data]).T
        packed_clusters = pack_clusters(data, winner_coordinates[1], 3)
        import_result_in_googlesheet(packed_clusters, worksheet, sigma, learning_rate)
        print(f"Данные в гугл-таблице на листе '{sheet_name}' обновлены!\n")
        while True:
          print("Попробовать изменить радиус и/или минамальное количество точек и попробовать снова?\n"
          "1. Да\n"
          "2. Нет\n")
          choice_2 = input("Введите опцию: ")
          if choice_2 == '1':
            break
          elif choice_2 == '2':
            break
          else:
            print("Я не знаю таких цифр")
        if choice_2 == '2':
            break
    elif choice == 3:
      print("ДРУГОЙ РЫЧАААГ")
      break
    else:
      print("Я не знаю таких цифр")
  except ValueError:
    print("Я не знаю таких цифр")

Выходные данные были обрезаны до нескольких последних строк (5000).
Я не знаю таких цифр
Я не знаю таких цифр
Я не знаю таких цифр
Я не знаю таких цифр
Я не знаю таких цифр
Я не знаю таких цифр
Я не знаю таких цифр
Я не знаю таких цифр
Я не знаю таких цифр
Я не знаю таких цифр
Я не знаю таких цифр
Я не знаю таких цифр
Я не знаю таких цифр
Я не знаю таких цифр
Я не знаю таких цифр
Я не знаю таких цифр
Я не знаю таких цифр
Я не знаю таких цифр
Я не знаю таких цифр
Я не знаю таких цифр
Я не знаю таких цифр
Я не знаю таких цифр
Я не знаю таких цифр
Я не знаю таких цифр
Я не знаю таких цифр
Я не знаю таких цифр
Я не знаю таких цифр
Я не знаю таких цифр
Я не знаю таких цифр
Я не знаю таких цифр
Я не знаю таких цифр
Я не знаю таких цифр
Я не знаю таких цифр
Я не знаю таких цифр
Я не знаю таких цифр
Я не знаю таких цифр
Я не знаю таких цифр
Я не знаю таких цифр
Я не знаю таких цифр
Я не знаю таких цифр
Я не знаю таких цифр
Я не знаю таких цифр
Я не знаю таких цифр
Я не знаю таких цифр
Я не зна

KeyboardInterrupt: 